## System Benchmarking Tool for Decentralized Energy Trading


### Imports

In [4]:
%%capture
import sys
import os
import math
!{sys.executable} -m pip intall web3
!{sys.executable} -m pip install matplotlib



### Configuration
- n - max. number of households to be benchmarked
- shareProducers - share of producing households


In [5]:
n = 5
shareProducers = 0.15


### Generate Contracts

In [ ]:
setup_time = []
witness_time = []
proof_time = []
constraints = []
%alias_magic benchmark timeit -p "-n 1 -r 1 -o"

def calcProducerConsumer(num):
    if math.ceil(num * shareProducers) > 1:
        wE = num * shareProducers
        nE = n - wE
    else:
        wE = 1
        nE = num - wE
    return wE, nE

# counts constraints by looking into out.ztf
def count_constraints():
    f = open("./out.ztf",'r')
    constraints.append(len(list(filter(lambda x: "==" in x, f.readlines()))))
    
def compile_file(file):
    value = %benchmark !zokrates compile -i settlement-check.zok --light
    return int(value.best * 1000000)

def setup():
    value = %benchmark !zokrates setup --light
    return int(value.best * 1000000)

%cd ..
for i in range(2, n + 1):
    wE, nE = calcProducerConsumer(i)
    %cd zokrates-code
    !node zoKratesCodeGenerator.js {wE} {nE}
    !zokrates compile -i settlement-check.zok --light
    !zokrates setup --light
    !zokrates export-verifier
    !pwd
    !cp ./verifier.sol ../contracts/verifier.sol
#     count_constraints()
    %cd ..
#     !truffle compile
    print("Compiling and deploying Contracts...")
    !truffle migrate --reset hard --network benchmark
    !node benchmark/setup_benchmark.js {wE} {nE}
%cd benchmark

Created `%benchmark` as an alias for `%timeit -n 1 -r 1 -o`.
Created `%%benchmark` as an alias for `%%timeit -n 1 -r 1 -o`.
/Users/paul/Documents/projects/decentralized-energy-trading
/Users/paul/Documents/projects/decentralized-energy-trading/zokrates-code
# of HHs with Energy:  1
# of HHs without Energy:  1
Generating zoKrates-Code for n = 1 HHs with Energy & m = 1 HHs without Energy and saving it to a file...
Generating the corresponding code for the Configuration of the NED-Server...
Saving the generated code to the ned-server-config.js File...
Generating the corresponding Solidity Contracts...
Saving the generated code to the corresponding Solidity-Contract Files...
Saving the generated code to the ned-server-config.js File...
Done!


%cd zokrates-code
!node zoKratesCodeGenerator.js {wE} {nE}